In [2]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  
import IPython.display as display

/var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/ipykernel_73235/4123656545.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [14]:
import speech_recognition as sr
import os 
import concurrent.futures
from tqdm import tqdm

In [6]:
def transcribe_wav(file_path):
    # Initialize recognizer
    recognizer = sr.Recognizer()
    
    # Load the audio file
    with sr.AudioFile(file_path) as source:
        # Listen to the audio file
        audio_data = recognizer.record(source)
        
        try:
            # Recognize speech using Google's Web Speech API
            text = recognizer.recognize_google(audio_data)
            print(f"Transcript: {text}")
            return text
        except sr.RequestError as e:
            print(f"Could not request results from Google Web Speech API; {e}")
        except sr.UnknownValueError:
            print("Google Web Speech API could not understand the audio")
            
if __name__ == "__main__":
    # Path to your .wav file
    wav_file_path = "/Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Prototype 4/male/battery/battery_sample_406.wav"
    
    # Transcribe the .wav file
    transcribe_wav_delete_matching(wav_file_path)

In [16]:
def transcribe_and_delete(file_path, phrase_to_match):
    recognizer = sr.Recognizer()
    
    try:
        # Load the audio file
        with sr.AudioFile(file_path) as source:
            audio_data = recognizer.record(source)
        
        # Recognize speech using Google's Web Speech API
        text = recognizer.recognize_google(audio_data)
        #print(f"Transcript for {file_path}: {text}")
        
        # Check if the transcript matches the given phrase
        if text == phrase_to_match:
            # Delete the file if it matches the phrase
            os.remove(file_path)
            #print(f"File {file_path} deleted because it matches the phrase '{phrase_to_match}'")
            return True
        else:
            #print(f"File {file_path} does not match the phrase '{phrase_to_match}'")
            return False
    except sr.RequestError as e:
        print(f"Could not request results from Google Web Speech API; {e}")
        return False
    except sr.UnknownValueError:
        print(f"Google Web Speech API could not understand the audio for {file_path}")
        return False

def transcribe_and_delete_files(source_folder, phrase_to_match, file_limit=50):
    # List all .wav files in the source folder
    files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.endswith(".wav")]
    
    # Initialize progress bar
    with tqdm(total=min(len(files), file_limit)) as pbar:
        # Use ThreadPoolExecutor for multithreading
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Track the number of deleted files
            delete_count = 0
            futures = {executor.submit(transcribe_and_delete, file, phrase_to_match): file for file in files}
            
            for future in concurrent.futures.as_completed(futures):
                if future.result():
                    delete_count += 1
                    pbar.update(1)
                
                if delete_count >= file_limit:
                    print(f"File limit of {file_limit} reached. Stopping deletion.")
                    break

if __name__ == "__main__":
    # Path to your source folder
    source_folder = "/Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Prototype 4/male/battery"
    
    # Phrase to match
    phrase_to_match = "how much charge does the battery have"
    
    # Transcribe and possibly delete files
    transcribe_and_delete_files(source_folder, phrase_to_match)

  0%|                                                                               | 0/50 [00:00<?, ?it/s]

Google Web Speech API could not understand the audio for /Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Prototype 4/male/battery/battery_sample_336.wav


  0%|                                                                               | 0/50 [00:24<?, ?it/s]
